In [1]:
import pymongo
import time
import pandas as pd
import json
import nltk

# Conexão com o MongoDB

In [4]:
client = pymongo.MongoClient()

# database
db = client['IWD2021']  

# collection
collection = db.tweets_pt

In [5]:
pipeline = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [6]:
time_start = time.time()
document = list(collection.aggregate(pipeline=pipeline))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
print(f'{len(document)} documentos selecionados')

Tempo de execução: 00:00:03
124444 documentos selecionados


In [7]:
print(f'Amostra de um documento:\n')
print(json.dumps(document[0], sort_keys=False, indent=3, ensure_ascii=False))

Amostra de um documento:

{
   "tweet_id": "1368641308982657025",
   "tweet_text": "🇧🇷 | Para celebrarmos o dia das mulheres, o @Fly923 estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.\n\n🔗: https://t.co/gL5nQXDBml https://t.co/6z6NZ4J4V6",
   "num_likes": 0,
   "num_retweets": 3,
   "quoted_status_id": "1368582569789448195",
   "retweeted_status_id": "1368592963304124416",
   "reply_to_user": null,
   "reply_to_status": null,
   "user_id": "207164069",
   "screen_name": "camisdjesus",
   "followers": 560,
   "following": 272,
   "date": "07/03/2021 19:14:50"
}


In [8]:
df = pd.DataFrame(document)
df = df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]
df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1368641308982657025,07/03/2021 19:14:50,"🇧🇷 | Para celebrarmos o dia das mulheres, o @F...",0,3,1368592963304124416,1368582569789448195,None,None,207164069,camisdjesus,560,272
1,1368643323246882816,07/03/2021 19:22:51,CAMPINAS NO MARÇO LILÁS\nMulheres de Campinas ...,7,2,None,None,None,None,176464755,CUTsaopaulo,6751,219
2,1368645634245398532,07/03/2021 19:32:02,"Em antecipação de #IWD2021 , conheça oito mulh...",1,0,None,1368638832036384771,None,None,322101708,a_b_Martins,211,1009
3,1368646590106370057,07/03/2021 19:35:49,"Mulheres na liderança! 🏅\n\nEm 1900, a tenista...",0,1,1368638363356565513,None,None,None,23601228,melangedetout,1996,725
4,1368649920094236672,07/03/2021 19:49:03,Bravo! 👏👏👏 https://t.co/ptfquSy3PO,4,0,None,1368630612488298503,None,None,2855855786,Sz3nna,494,633


In [9]:
print(f'Amostra de um tweet antes do pré-processamento:\n\n{df.tweet_text[0]}')

Amostra de um tweet antes do pré-processamento:

🇧🇷 | Para celebrarmos o dia das mulheres, o @Fly923 estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.

🔗: https://t.co/gL5nQXDBml https://t.co/6z6NZ4J4V6


In [10]:
df.to_csv('tweets_iwd2021_pt.csv')

## Pré-processamento

In [11]:
# remover links
df['clean_text'] = df['tweet_text'].str.replace(r"http\S+","") 
df.clean_text[0]

'🇧🇷 | Para celebrarmos o dia das mulheres, o @Fly923 estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.\n\n🔗:  '

In [12]:
# remover mentions
df['clean_text'] = df['clean_text'].str.replace(r"@\S+","")
df.clean_text[0]

'🇧🇷 | Para celebrarmos o dia das mulheres, o  estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.\n\n🔗:  '

In [13]:
# remover hashtags
df['clean_text'] = df['clean_text'].str.replace(r"#\Sigma+"," ")
df.clean_text[0]

'🇧🇷 | Para celebrarmos o dia das mulheres, o  estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.\n\n🔗:  '

In [14]:
# remover quebra de linhas
df['clean_text'] = df['clean_text'].str.replace(r"\n"," ")
df.clean_text[0]

'🇧🇷 | Para celebrarmos o dia das mulheres, o  estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.  🔗:  '

In [15]:
# remover pontuações
df['clean_text'] = df['clean_text'].str.replace(r"[^\w\s#]|_"," ")
df.clean_text[0]

'     Para celebrarmos o dia das mulheres  o  estará tocando as nossas artistas femininas favoritas o dia todo  Entre no site abaixo e solicite Something to Believe In de Laura Marano       '

In [16]:
# remover números
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\b(?:[0-9]*)\b","")
df.clean_text[0]

'Para celebrarmos o dia das mulheres  o  estará tocando as nossas artistas femininas favoritas o dia todo  Entre no site abaixo e solicite Something to Believe In de Laura Marano'

In [17]:
# remover espaços duplos
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\s{2,}"," ")
df.clean_text[0]

'Para celebrarmos o dia das mulheres o estará tocando as nossas artistas femininas favoritas o dia todo Entre no site abaixo e solicite Something to Believe In de Laura Marano'

In [18]:
# converter todas as letras para minusculas
df['clean_text'] = df['clean_text'].str.lower()
df.clean_text[0]

'para celebrarmos o dia das mulheres o estará tocando as nossas artistas femininas favoritas o dia todo entre no site abaixo e solicite something to believe in de laura marano'

In [19]:
# remover letras que se repetem em sequência mais de 3x
import string
alphabet = list(string.ascii_lowercase)

for letter in alphabet:
    pattern = '{}{}'.format(letter, '{3,}')
    df['clean_text'] = df['clean_text'].str.replace(r'{}'.format(pattern), letter)
    
df.clean_text[0]

'para celebrarmos o dia das mulheres o estará tocando as nossas artistas femininas favoritas o dia todo entre no site abaixo e solicite something to believe in de laura marano'

In [20]:
# remover stpwords
with open('../stopwords/portuguese.txt', 'r') as file:
    stopwords = file.readlines()
stopwords = [sw.replace('\n','') for sw in stopwords]

def remover_stopwords(texto):
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_stopwords)
df.clean_text[0]

'celebrarmos dia mulheres estará tocando artistas femininas favoritas dia site abaixo solicite something to believe in laura marano'

In [21]:
# remover palavras com 2 caracteres ou menos
def remover_len_2(texto):
    palavras = [i for i in texto.split() if len(i) > 2]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_len_2)
df.clean_text[0]

'celebrarmos dia mulheres estará tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano'

In [22]:
print(
    f'Amostra de um tweet depois do pré-processamento:\n\n{df.tweet_text[0]}\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

🇧🇷 | Para celebrarmos o dia das mulheres, o @Fly923 estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.

🔗: https://t.co/gL5nQXDBml https://t.co/6z6NZ4J4V6

celebrarmos dia mulheres estará tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano


In [23]:
def reduzir_radical(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = [stemmer.stem(i) for i in texto.split()]
    return (" ".join(palavras))

In [24]:
reduzir_radical(df.clean_text[0])

'celebr dia mulh est toc artist feminin favorit dia sit abaix solicit something believ laur maran'

In [25]:
# df['tweet_text'] = df['tweet_text'].apply(reduzir_radical)

In [26]:
import unicodedata

def remover_acentuacao(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return (palavra_sem_acento)

In [27]:
remover_acentuacao(df.clean_text[0])

'celebrarmos dia mulheres estara tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano'

In [28]:
df['clean_text'] = df['clean_text'].apply(remover_acentuacao)
df.clean_text[0]

'celebrarmos dia mulheres estara tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano'

In [29]:
# remover non ascii caracteres
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"[^a-zA-z0-9#|\s]","")

df.clean_text[0]

'celebrarmos dia mulheres estara tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano'

In [30]:
print(
    f'Amostra de um tweet depois do pré-processamento:\n\n{df.tweet_text[0]}\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

🇧🇷 | Para celebrarmos o dia das mulheres, o @Fly923 estará tocando as nossas artistas femininas favoritas o dia todo. Entre no site abaixo e solicite Something to Believe In de Laura Marano.

🔗: https://t.co/gL5nQXDBml https://t.co/6z6NZ4J4V6

celebrarmos dia mulheres estara tocando artistas femininas favoritas dia site abaixo solicite something believe laura marano


### Etapa opcional

O atributo 'lang' presentes nos documentos podem ter sido atribuidos erroneamente. Objetivando eliminar tweets que possam ser de um idioma não requisitado, desenvolveram-se 2 funções para ajudar a confirmar o idioma destes.

Para desabilitar warnings:

In [ ]:
# import logging
# logging.disable(logging.WARNING)

#### 1. Usando polyglot

In [ ]:
# import polyglot
# from polyglot.detect import Detector

In [ ]:
# def get_polyglot_language(text):
#     try:
#         detector = Detector(text)
#     except polyglot.detect.base.UnknownLanguage:
#         lang = 'Undefined'
#     else:
#         lang = detector.language.name
    
#     return (lang)

In [ ]:
# get_polyglot_language(df['tweet_text'][0])

In [ ]:
# df['polyglot_lang'] = df['tweet_text'].apply(get_polyglot_language)

In [ ]:
# df['polyglot_lang'].value_counts()

#### 2. Usando textblob

In [ ]:
# import textblob
# from textblob import TextBlob
# import pycountry

In [ ]:
# def get_textblob_language(text):
#     try:
#         time.sleep(0.2)
#         b = TextBlob(text)
#         language = pycountry.languages.get(alpha_2=b.detect_language())
#         language_name = language.name
#     except textblob.exceptions.TranslatorError:
#         # language_name = "Deu ruim"
#         language_name = 'Undefined'
    
#     return(language_name) 

In [ ]:
# get_textblob_language(df['tweet_text'][0])

In [ ]:
# df['textblob_lang'] = df['tweet_text'].apply(get_textblob_language)

Para reabilitar warnings:

In [ ]:
# logging.disable(logging.NOTSET)

In [ ]:
# df.shape

# Criação do arquivo .csv

### Cenário 1: base completa

Caso tenha se utilizado umas das duas funções - **get_polyglot_language**, por exemplo - use o seguinte trecho para gerar o arquivo .csv:

In [ ]:
# index = df['polyglot_lang'] == 'Portuguese'
# new_df = df[index]
# new_df = new_df.drop('polyglot_lang', axis=1)

In [ ]:
# new_df.to_csv('cenario1_8M2020_polyglot_tweets_pt.csv', index=False)

Caso contrário:

In [ ]:
# df = df.drop('polyglot_lang', axis=1)

In [31]:
df = df[df['clean_text'] != '']

In [32]:
df.shape

(124293, 14)

In [33]:
df.to_csv('cenario1_8M2021_tweets_pt.csv', index=False)

criação do arquivo .txt (somente com os textos dos tweets):

In [34]:
import numpy as np

In [35]:
tweets_text = np.array(df['clean_text'])

In [36]:
np.savetxt(fname='cenario1_8M2021_tweets_pt.txt', X=tweets_text, fmt='%s')

## Cenário 2: tweets sem duplicatas

In [37]:
df.drop_duplicates(['clean_text'], inplace = True)

/home/gean/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
df.shape

(33438, 14)

In [39]:
df.to_csv('cenario2_8M2021_tweets_pt.csv', index=False)

In [40]:
tweets_text = np.array(df['clean_text'])

In [41]:
np.savetxt(fname='cenario2_8M2021_tweets_pt.txt', X=tweets_text, fmt='%s')